<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Wavegan-drum-dataset" data-toc-modified-id="Wavegan-drum-dataset-0.1"><span class="toc-item-num">0.1&nbsp;&nbsp;</span>Wavegan drum dataset</a></span></li><li><span><a href="#Musicradar-essential-drumkit" data-toc-modified-id="Musicradar-essential-drumkit-0.2"><span class="toc-item-num">0.2&nbsp;&nbsp;</span>Musicradar essential drumkit</a></span></li><li><span><a href="#Musicradar-1000" data-toc-modified-id="Musicradar-1000-0.3"><span class="toc-item-num">0.3&nbsp;&nbsp;</span>Musicradar 1000</a></span></li><li><span><a href="#200-Drum-Machines" data-toc-modified-id="200-Drum-Machines-0.4"><span class="toc-item-num">0.4&nbsp;&nbsp;</span>200 Drum Machines</a></span></li><li><span><a href="#MDLib2.2" data-toc-modified-id="MDLib2.2-0.5"><span class="toc-item-num">0.5&nbsp;&nbsp;</span>MDLib2.2</a></span></li><li><span><a href="#Fraunhofer-IDMT-SMT" data-toc-modified-id="Fraunhofer-IDMT-SMT-0.6"><span class="toc-item-num">0.6&nbsp;&nbsp;</span>Fraunhofer IDMT SMT</a></span></li><li><span><a href="#Hydrogen-main-sound-libraries" data-toc-modified-id="Hydrogen-main-sound-libraries-0.7"><span class="toc-item-num">0.7&nbsp;&nbsp;</span>Hydrogen main sound libraries</a></span></li></ul></li><li><span><a href="#Wavegan" data-toc-modified-id="Wavegan-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Wavegan</a></span></li><li><span><a href="#Musicradar" data-toc-modified-id="Musicradar-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Musicradar</a></span></li><li><span><a href="#MDLib" data-toc-modified-id="MDLib-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>MDLib</a></span><ul class="toc-item"><li><span><a href="#Part-1" data-toc-modified-id="Part-1-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Part 1</a></span></li><li><span><a href="#Part-2" data-toc-modified-id="Part-2-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Part 2</a></span></li></ul></li><li><span><a href="#Hydrogen-Main-Library" data-toc-modified-id="Hydrogen-Main-Library-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Hydrogen Main Library</a></span></li><li><span><a href="#Musicradar-1000" data-toc-modified-id="Musicradar-1000-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Musicradar 1000</a></span></li><li><span><a href="#Merge-datasets" data-toc-modified-id="Merge-datasets-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Merge datasets</a></span></li><li><span><a href="#Cleaning" data-toc-modified-id="Cleaning-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Cleaning</a></span><ul class="toc-item"><li><span><a href="#Anomalies" data-toc-modified-id="Anomalies-7.1"><span class="toc-item-num">7.1&nbsp;&nbsp;</span>Anomalies</a></span></li></ul></li></ul></div>

In [105]:
import glob
import os
import pandas as pd
from pandas import DataFrame
import umap
from librosa import feature
from librosa.util import fix_length
from librosa.core import resample, load
import timbral_models
import numpy as np
import IPython

In [2]:
from tools import display_embedding

### Wavegan drum dataset
http://deepyeti.ucsd.edu/cdonahue/wavegan/data/drums.tar.gz
### Musicradar essential drumkit
http://cdn.mos.musicradar.com/audio/samples/musicradar-essential-drumkit-samples.zip
### Musicradar 1000
https://www.musicradar.com/news/drums/1000-free-drum-samples
### 200 Drum Machines
http://www.hexawe.net/mess/200.Drum.Machines/
### MDLib2.2
Articulation dataset

http://www.mattprockup.com/percussion-dataset

https://www.dropbox.com/s/p736vokha3240e6/MDLib2.2.zip?dl=0
### Fraunhofer IDMT SMT

https://www.idmt.fraunhofer.de/en/business_units/m2d/smt/drums.html

### Hydrogen main sound libraries
https://sourceforge.net/projects/hydrogen/files/Sound%20Libraries/Main%20sound%20libraries/

In [3]:
base_dir = "./Samples"
!ls $base_dir

 200-drum-machines	  MDLib2.2
'Freesound Percussions'   musicradar-drum-samples
 Hydrogen		  musicradar-essential-drumkit-samples
 IDMT-SMT-DRUMS-V2	  wavegan-drums


In [4]:
!ls {base_dir}/MDLib2.2/Sorted/*

./Samples/MDLib2.2/Sorted/Cymbals:
'Bright Crash'	'Dark Crash'   HiHat   Ride

'./Samples/MDLib2.2/Sorted/Floor Tom':
Buzz  Rim  Strike

'./Samples/MDLib2.2/Sorted/Floor Tom SNoff':
Buzz  Rim  Strike

./Samples/MDLib2.2/Sorted/Kick:
'Dead Kick'  'Long Kick'

'./Samples/MDLib2.2/Sorted/Kick SNoff':
'Dead Kick'  'Long Kick'

'./Samples/MDLib2.2/Sorted/Rack Tom':
Buzz  Rim  Strike

'./Samples/MDLib2.2/Sorted/Rack Tom SNoff':
Buzz  Rim  Strike

./Samples/MDLib2.2/Sorted/Snare:
 Buzz  'Cross Stick'   Rim   Strike

'./Samples/MDLib2.2/Sorted/Snare SNoff':
 Buzz  'Cross Stick'   Rim   Strike


In [5]:
wavegan = glob.glob(base_dir + '/wavegan-drums/*/*.wav')
musicradar = glob.glob(base_dir + '/musicradar-essential-drumkit-samples/*/*.wav')
drum200 = glob.glob(base_dir + '/200-drum-machines/*/*.wav')
mdlib1 = glob.glob(base_dir + '/MDLib2.2/Sorted/*/*/*.wav')
mdlib2 = glob.glob(base_dir + '/MDLib2.2/Sorted/*/*/*/*.wav')
#idmt = glob.glob(base_dir + '/IDMT-SMT-DRUMS-V2/audio/*.wav')
hydrogen = glob.glob(base_dir + '/Hydrogen/*/*.wav')
hydrogen.extend(glob.glob(base_dir + '/Hydrogen/*/*.flac'))
musicradar1000 = glob.glob(base_dir + '/musicradar-drum-samples/*/*.wav')
musicradar1000.extend(glob.glob(base_dir + '/musicradar-drum-samples/*/*/*.wav'))

In [6]:
def split(level=1):
    def _split(row):
        directory = row['path']
        for lvl in range(level):
            if lvl == 0:
                colname = 'filename'
            else:
                colname = 'dir_' + str(lvl)
            directory, row[colname] = os.path.split(directory)
        return row
    return _split

In [7]:
def set_classes(df, column, rules):
    for d_class, pattern in rules.items():
        mask = df[column].str.contains(pattern, case=False)
        df.loc[mask, d_class] = 1
        df.loc[~mask, d_class] = 0

## Wavegan

In [82]:
wavegan_df = DataFrame(data=wavegan, columns=['path'])
wavegan_df = wavegan_df.apply(split(),axis=1)
wavegan_df

,path,filename
0,./Samples/wavegan-drums/valid/Tom_00041.wav,Tom_00041.wav
1,./Samples/wavegan-drums/valid/Snare_00133.wav,Snare_00133.wav
2,./Samples/wavegan-drums/valid/Hat_00049.wav,Hat_00049.wav
3,./Samples/wavegan-drums/valid/Snare_00179.wav,Snare_00179.wav
4,./Samples/wavegan-drums/valid/Tom_00343.wav,Tom_00343.wav
...,...,...
2997,./Samples/wavegan-drums/train/Tom_00655.wav,Tom_00655.wav
2998,./Samples/wavegan-drums/train/Clap_00886.wav,Clap_00886.wav
2999,./Samples/wavegan-drums/train/Tom_00803.wav,Tom_00803.wav
3000,./Samples/wavegan-drums/train/Snare_00743.wav,Snare_00743.wav


In [85]:
rules = {
    'class_Tom': 'Tom',
    'class_Snare': 'Snare',
    'class_Hat': 'Hat',
    'class_Clap': 'Clap',
    'class_Cowbell': 'Cowbell',
    'class_Cymbal': 'Ride|Crash',
    'subclass_Cymbal_Crash': 'Crash',
    'subclass_Cymbal_Ride': 'Ride',
    'class_Kick': 'Kick',
        }

set_classes(wavegan_df, 'filename', rules)
wavegan_df['dataset'] = 'Wavegan'

In [86]:
mask = wavegan_df.loc[:, [c for c in wavegan_df.columns if c.startswith('class_')] ].sum(axis=1) != 1
print('unlabelled:', len(wavegan_df[mask]))
print('labelled:', len(wavegan_df[~mask]))
print(list(wavegan_df[mask]['filename']))

unlabelled: 0
labelled: 3002
[]


## Musicradar

In [87]:
musicradar_df = DataFrame(data=musicradar, columns=['path'])
musicradar_df = musicradar_df.apply(split(2),axis=1)
musicradar_df

,path,filename,dir_1
0,./Samples/musicradar-essential-drumkit-samples...,TR 808 Closed Hat 01.wav,TR 808 Kit
1,./Samples/musicradar-essential-drumkit-samples...,TR 808 Closed Hat 02.wav,TR 808 Kit
2,./Samples/musicradar-essential-drumkit-samples...,TR 808 Snare 01.wav,TR 808 Kit
3,./Samples/musicradar-essential-drumkit-samples...,TR 808 Rimshot.wav,TR 808 Kit
4,./Samples/musicradar-essential-drumkit-samples...,TR 808 Cymbal 01.wav,TR 808 Kit
...,...,...,...
489,./Samples/musicradar-essential-drumkit-samples...,D&B Drop FX 01.wav,Drum & Bass Kit
490,./Samples/musicradar-essential-drumkit-samples...,D&B Snare 03.wav,Drum & Bass Kit
491,./Samples/musicradar-essential-drumkit-samples...,D&B Kick 03.wav,Drum & Bass Kit
492,./Samples/musicradar-essential-drumkit-samples...,D&B Kick 01.wav,Drum & Bass Kit


In [91]:
rules ={
    'class_Tom': 'Tom',
    'class_Snare': 'Snare|rim',
    'class_Kick': 'Kick',
    'subclass_Snare_Rim': 'rim',
    'class_Shaken': 'shake[nr]|mara?cas|cabas',
    'subclass_Shaken_Shaker': 'shaker',
    'class_Tambourin': 'tamb',
    'subclass_Shaken_Maracas': 'mara?cas',
    'subclass_Shaken_Cabasa': 'cabas',
    'class_Cymbal': 'Cymbal',
    'class_Conga': 'Conga',
    'class_Bongo': 'Bongo',
    'class_Djembe': 'Djmbe',
    'class_Tabla': 'Tabla',
    'class_Darbuka': 'Darbuka',
    'class_Cajon': 'Cajon',
    'class_Timbales': 'timb',
    'class_Hat': 'hat',
    'subclass_Hat_Open': 'open hat',
    'subclass_Hat_Close': 'closed? hat',
    'class_Clap': 'clap',
    'class_Cowbell': 'cowbell',
    'class_Triangle': 'triangle',
    'class_Agogo': 'agogo',
    'class_clave': 'clave',
       }

set_classes(musicradar_df, 'filename', rules)
musicradar_df['dataset'] = 'Musicradar'

mask = musicradar_df.loc[:, [c for c in musicradar_df.columns if c.startswith('class_')] ].sum(axis=1) != 1
print('unlabelled:', len(musicradar_df[mask]))
print('labelled:', len(musicradar_df[~mask]))
print(list(musicradar_df[mask]['filename']))

unlabelled: 70
labelled: 424
['Modular Perc 01.wav', 'Modular Perc 03.wav', 'Modular Perc 04.wav', 'Modular Perc 05.wav', 'Modular Perc 06.wav', 'Modular Perc 02.wav', 'Foley Bucket Perc.wav', 'Foley Can Perc.wav', 'Foley Tube Perc.wav', 'Foley Paint Tin Perc.wav', 'Vintage Perc 01.wav', 'Vintage Perc 04.wav', 'Vintage Perc 03.wav', 'Vintage Perc 02.wav', 'ChipTune Arcade FX 06.wav', 'ChipTune Perc 05.wav', 'ChipTune Arcade FX 08.wav', 'ChipTune Arcade FX 01.wav', 'ChipTune Perc 01.wav', 'ChipTune Arcade FX 02.wav', 'ChipTune Perc 03.wav', 'ChipTune Perc 04.wav', 'ChipTune Arcade FX 03.wav', 'ChipTune Arcade FX 04.wav', 'ChipTune Perc 02.wav', 'ChipTune Arcade FX 05.wav', 'ChipTune Arcade FX 07.wav', '80s Digital Bonga 01.wav', 'Vermona Fx 05.wav', 'Vermona Perc 01.wav', 'Vermona Perc 09.wav', 'Vermona Perc 04.wav', 'Vermona Perc 05.wav', 'Vermona Fx 03.wav', 'Vermona Perc 06.wav', 'Vermona Fx 04.wav', 'Vermona Perc 02.wav', 'Vermona Fx 01.wav', 'Vermona Perc 08.wav', 'Vermona Perc 07.

In [92]:
shaken_classes = [c for c in musicradar_df.columns if c.startswith('subclass_Shaken')]
mask = (musicradar_df['class_Shaken'] == 1) ^ (musicradar_df[shaken_classes] == 1).any(axis=1)
print('mislabelled:\n', musicradar_df[mask][['filename', 'class_Shaken'] + shaken_classes])
print('mislabelled:\n', list(musicradar_df[mask]['filename']))

mislabelled:
                  filename  class_Shaken  subclass_Shaken_Shaker  \
382    Shaken Tamb 05.wav           1.0                     0.0   
383  Shaken Rainstick.wav           1.0                     0.0   
387   Shaken Bells 01.wav           1.0                     0.0   
388    Shaken Tamb 02.wav           1.0                     0.0   
390      Shaken Gourd.wav           1.0                     0.0   
392    Shaken Jingles.wav           1.0                     0.0   
395    Shaken Tamb 04.wav           1.0                     0.0   
396    Shaken Tamb 03.wav           1.0                     0.0   
397    Shaken Shekere.wav           1.0                     0.0   
399   Shaken Bells 02.wav           1.0                     0.0   
400    Shaken Tamb 01.wav           1.0                     0.0   
402    Shaken Caxixi .wav           1.0                     0.0   

     subclass_Shaken_Maracas  subclass_Shaken_Cabasa  
382                      0.0                     0.0  
383 

In [14]:
## Drum200

In [95]:
drum200_df = DataFrame(data=drum200, columns=['path'])
drum200_df = drum200_df.apply(split(2),axis=1)
drum200_df

,path,filename,dir_1
0,./Samples/200-drum-machines/Yamaha RX-11/MaxV ...,MaxV - RX11 OPHAT2.wav,Yamaha RX-11
1,./Samples/200-drum-machines/Yamaha RX-11/MaxV ...,MaxV - RX11 BD3.wav,Yamaha RX-11
2,./Samples/200-drum-machines/Yamaha RX-11/MaxV ...,MaxV - RX11 BD2.wav,Yamaha RX-11
3,./Samples/200-drum-machines/Yamaha RX-11/MaxV ...,MaxV - RX11 CLHAT2.wav,Yamaha RX-11
4,./Samples/200-drum-machines/Yamaha RX-11/MaxV ...,MaxV - RX11 RIDE.wav,Yamaha RX-11
...,...,...,...
6745,./Samples/200-drum-machines/Korg SR-120/MaxV -...,MaxV - HH Op.wav,Korg SR-120
6746,./Samples/200-drum-machines/Korg SR-120/KORG S...,KORG SR120 8.wav,Korg SR-120
6747,./Samples/200-drum-machines/Korg SR-120/KORG S...,KORG SR120 6.wav,Korg SR-120
6748,./Samples/200-drum-machines/Korg SR-120/KORG S...,KORG SR12011.wav,Korg SR-120


In [98]:
rules = {
    'class_Hat': 'hat|hh|close|open hi',
    'subclass_Hat_Open': 'op?.?(hi)?hat|hat.?o|close|pedal|c.?hh|hh.?c|foot|open hi',
    'subclass_Hat_Close': '((cl?)|p).?(hi)?hat|hat.?(c|p)|ope?n|o.?hh|hh.?o',
    'class_Bongo': 'Bongo',
    'class_Cymbal': 'cym|cra?sh|ride|chi?n[^o]|spla?sh',
    'subclass_Cymbal_Crash': 'cra?sh',
    'subclass_Cymbal_Ride': 'ride',
    'subclass_Cymbal_Chinese': 'chi?n[^o]',
    'subclass_Cymbal_Splash': 'spla?sh',
    'class_Tom': 'tom',
    'class_Conga': 'cong',
    'class_Kick': 'kic?k|bd',
    'class_Gong': 'Gong',
    'class_Cowbell': 'cow|Cowbell',
    'class_Snare': 'sna?r|rim',
    'subclass_Snare_Rim': 'rim',
    'class_Clap': 'clap',
    'class_Shaken': 'shake|cabas',
    'subclass_Shaken_Shaker': 'shaker',
    'class_Tambourin': 'tamb',
    'subclass_Shaken_Cabasa': 'cabas',
    'class_Clave': 'cla?ve',
    'class_Timpani': 'timp',
    'class_Timbales': 'timb',
    'class_Agogo': 'agogo',
    'class_Triangle': 'triangle',
    'class_Djembe': 'djembe',
    'class_Tabla': 'tabla',
    'class_Darbuka': 'darbuka',
        }

set_classes(drum200_df, 'filename', rules)
drum200_df['dataset'] = 'Drum200'

In [99]:
mask = drum200_df.loc[:, [c for c in drum200_df.columns if c.startswith('class_')] ].sum(axis=1) < 1
print('unlabelled:', len(drum200_df[mask]))
print('labelled:', len(drum200_df[~mask]))
print('unlabelled:', list(drum200_df[mask]['filename']))

unlabelled: 3461
labelled: 3289
unlabelled: ['MaxV - RX11 MED SN.wav', 'MaxV - RX11 LITE SN.wav', 'MaxV - RX11 HEVY SN.wav', 'MaxV - Metal.wav', '13.wav', '06.wav', '05.wav', '09.wav', '08.wav', '01.wav', '00.wav', '14.wav', '11.wav', '02.wav', '12.wav', '03.wav', '10.wav', '04.wav', '07.wav', '15.wav', 'SR16Tank.wav', 'SR16FSnap.wav', 'SR16Guiro.wav', 'SR16Spark.wav', 'SR16Block.wav', 'Acoustic Bass Drum.wav', 'MaxV - EMU14.wav', 'MaxV - KDR21.wav', 'MaxV - EMU13.wav', 'MaxV - EMU20.wav', 'MaxV - EMU19.wav', 'MaxV - DRM03.wav', 'MaxV - SND24.wav', 'MaxV - EMU15.wav', 'MaxV - EMU09.wav', 'MaxV - DRM04.wav', 'MaxV - EMU12.wav', 'MaxV - CLD01.wav', 'MaxV - DRM08.wav', 'MaxV - DRM07.wav', 'MaxV - EMU17.wav', 'MaxV - OHD22.wav', 'MaxV - EMU16.wav', 'MaxV - EMU18.wav', 'MaxV - EMU_SD.wav', 'MaxV - DRM05.wav', 'MaxV - EMU11.wav', 'DrumulatorBlock.wav', 'MaxV - EMU10.wav', 'MaxV - DRM06.wav', 'MaxV - SND23.wav', 'MaxV - CLD02.wav', 'MaxV - Snap.wav', '06.wav', '05.wav', '01.wav', '00.wav', '0

In [63]:
mask = drum200_df.loc[:, [c for c in drum200_df.columns if c.startswith('class_')] ].sum(axis=1) > 1
print('overlabelled:', len(drum200_df[mask]))
# display('overlabelled:', drum200_df[mask])
print('overlabelled:', list(drum200_df[mask]['filename']))

overlabelled: 15
overlabelled: ['MaxV - HHTamb.wav', 'MaxV - ATOMBDEN.wav', 'MaxV - ATOMBDRV.wav', 'MaxV - ATOMBDGA.wav', 'MaxV - D#4 Open High Conga.wav', 'TOMSnare.wav', 'TOMHat_O.wav', 'TOMClap.wav', 'TOMCymbal.wav', 'TOMKick.wav', 'TOMHat_C.wav', 'MaxV - RIM-TOM3.wav', 'MaxV - RIM-TOM1.wav', 'MaxV - RIM-TOM2.wav', 'MaxV - RIM-TOM4.wav']


In [19]:
mask = (drum200_df['class_Hat'] == 1) ^ ( (drum200_df['subclass_Hat_Open'] ==1) | (drum200_df['subclass_Hat_Close'] ==1 ))
print('mislabelled:\n', drum200_df[mask][['filename', 'class_Hat', 'subclass_Hat_Open', 'subclass_Hat_Close']])
print('mislabelled:\n', list(drum200_df[mask]['filename']))

mislabelled:
                          filename  class_Hat  subclass_Hat_Open  \
196          MaxV - HiHat Mid.wav        1.0                0.0   
280                 MaxV - hh.wav        1.0                0.0   
414           MaxV - Opencuic.wav        0.0                0.0   
455           MaxV - Openpand.wav        0.0                0.0   
461           MaxV - Opensurd.wav        0.0                0.0   
...                           ...        ...                ...   
6585  MaxV - Roland -_open 29.wav        0.0                0.0   
6613  MaxV - Roland -open3 19.wav        0.0                0.0   
6653     MaxV - Mattel Hi-Hat.wav        1.0                0.0   
6721           MaxV - 808OPEN.wav        0.0                0.0   
6722          MaxV - 808OPEN_.wav        0.0                0.0   

      subclass_Hat_Close  
196                  0.0  
280                  0.0  
414                  1.0  
455                  1.0  
461                  1.0  
...                

In [20]:
mask = (drum200_df['class_Cymbal'] == 1) ^ ((drum200_df['subclass_Cymbal_Crash'] ==1) |
                                             (drum200_df['subclass_Cymbal_Ride'] ==1) |
                                             (drum200_df['subclass_Cymbal_Chinese'] ==1) |
                                             (drum200_df['subclass_Cymbal_Splash'] ==1) 
                                             )
#print('unlabelled:\n', drum200_df[mask][['filename', 'class_Cymbale', 'subclass_Cymbale_Crash', 'subclass_Cymbale_Ride']])
print('mislabeled:', len(drum200_df[mask]['filename']))
print(list(drum200_df[mask]['filename']))

mislabeled: 47
['DR-110Cymbal.wav', 'MaxV - EMU_CYMB.wav', 'TR606Cymb04.wav', 'TR606Cymb01.wav', 'TR606Cymb_OD.wav', 'TR606Cymb05.wav', 'TR606Cymb02.wav', 'TR606Cymb03.wav', 'MaxV - Cymbal.wav', 'MaxV - Cymbal.wav', '512cymb.wav', 'CR8000Cymbal.wav', 'MR10Cymbal.wav', 'MaxV - Cymshort.wav', 'MaxV - Cym2.wav', 'MaxV - Cym1.wav', 'MaxV - Cym3.wav', 'Cymbal2.wav', 'Cymbal1.wav', 'MaxV - 110_cymbal.wav', 'Ddm110Cymbal.wav', 'MaxV - Cymbal.wav', 'MaxV - Cymbal.wav', 'MaxV - Cymbal.wav', 'R8FingerCym.wav', 'R8CymbHand.wav', 'R-5 Back Cymbal.wav', 'MaxV - Cymbal.wav', 'TOMCymbal.wav', '202cym08.wav', '202cym03.wav', '202cym07.wav', '202cym06.wav', '202cym01.wav', '202cym04.wav', '202cym05.wav', '202cym02.wav', '202cym10.wav', '202cym09.wav', 'MaxV - Cymbal.wav', 'MaxV - locymbal.wav', 'MaxV - hicymbal.wav', 'DR220eCymbal.wav', 'MaxV - Cymbal.wav', 'KPRCymbal.wav', 'MaxV - Mattel Cymbal.wav', 'MaxV - Cymbal.wav']


In [21]:
drum200_df[drum200_df.iloc[:,4:].sum(axis=1) > 1].iloc[:,5:]

,subclass_Hat_Close,class_Bongo,class_Cymbal,subclass_Cymbal_Crash,subclass_Cymbal_Ride,subclass_Cymbal_Chinese,subclass_Cymbal_Splash,class_Tom,class_Conga,class_Kick,...,class_Tambourin,subclass_Shaken_Cabasa,class_Clave,class_Timpani,class_Agogo,class_Triangle,class_Djembe,class_Tabla,class_Darbuka,dataset
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Drum200
4,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Drum200
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Drum200
10,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Drum200
15,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Drum200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6680,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Drum200
6684,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Drum200
6685,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Drum200
6686,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Drum200


## MDLib

### Part 1

In [22]:
mdlib1_df = DataFrame(data=mdlib1, columns=['path'])
mdlib1_df = mdlib1_df.apply(split(3),axis=1)
mdlib1_df

,path,filename,dir_1,dir_2
0,./Samples/MDLib2.2/Sorted/Snare SNoff/Buzz/ST_...,ST_Snare_Buzz_0112.2.wav,Buzz,Snare SNoff
1,./Samples/MDLib2.2/Sorted/Snare SNoff/Buzz/DI_...,DI_Snare_Buzz_0224.3.wav,Buzz,Snare SNoff
2,./Samples/MDLib2.2/Sorted/Snare SNoff/Buzz/IN_...,IN_Snare_Buzz_0214.1.wav,Buzz,Snare SNoff
3,./Samples/MDLib2.2/Sorted/Snare SNoff/Buzz/ST_...,ST_Snare_Buzz_0333.3.wav,Buzz,Snare SNoff
4,./Samples/MDLib2.2/Sorted/Snare SNoff/Buzz/MN_...,MN_Snare_Buzz_0111.4.wav,Buzz,Snare SNoff
...,...,...,...,...
7483,./Samples/MDLib2.2/Sorted/Floor Tom/Strike/DI_...,DI_Floor Tom_Strike_1124.2.wav,Strike,Floor Tom
7484,./Samples/MDLib2.2/Sorted/Floor Tom/Strike/MN_...,MN_Floor Tom_Strike_1113.2.wav,Strike,Floor Tom
7485,./Samples/MDLib2.2/Sorted/Floor Tom/Strike/IN_...,IN_Floor Tom_Strike_1113.2.wav,Strike,Floor Tom
7486,./Samples/MDLib2.2/Sorted/Floor Tom/Strike/MN_...,MN_Floor Tom_Strike_1322.4.wav,Strike,Floor Tom


In [23]:
mdlib1_df['dir_1'].unique() , mdlib1_df['dir_2'].unique()

(array(['Buzz', 'Rim', 'Strike', 'Cross Stick'], dtype=object),
 array(['Snare SNoff', 'Rack Tom SNoff', 'Floor Tom SNoff', 'Snare',
        'Rack Tom', 'Floor Tom'], dtype=object))

In [24]:
rules = {
    'class_Snare': 'Snare',
    'class_Tom': 'Tom',
        }

set_classes(mdlib1_df, 'dir_2', rules)
mdlib1_df['dataset'] = 'MDLib'

In [25]:
mask = mdlib1_df.loc[:, [c for c in mdlib1_df.columns if c.startswith('class_')] ].sum(axis=1) < 1
print('unlabelled:', len(mdlib1_df[mask]))
print('labelled:', len(mdlib1_df[~mask]))
print('unlabelled:', list(mdlib1_df[mask]['dir_2']))

unlabelled: 0
labelled: 7488
unlabelled: []


In [61]:
mask = mdlib1_df.loc[:, [c for c in mdlib1_df.columns if c.startswith('class_')] ].sum(axis=1) > 1
print('overlabelled:', len(mdlib1_df[mask]))
print('overlabelled:', list(mdlib1_df[mask]['filename']))

overlabelled: 0
overlabelled: []


### Part 2

In [26]:
mdlib2_df = DataFrame(data=mdlib2, columns=['path'])
mdlib2_df = mdlib2_df.apply(split(4),axis=1)
mdlib2_df

,path,filename,dir_1,dir_2,dir_3
0,./Samples/MDLib2.2/Sorted/Cymbals/Bright Crash...,ST_Bright Crash_Crash_1112.3.wav,Crash,Bright Crash,Cymbals
1,./Samples/MDLib2.2/Sorted/Cymbals/Bright Crash...,DI_Bright Crash_Crash_1112.3.wav,Crash,Bright Crash,Cymbals
2,./Samples/MDLib2.2/Sorted/Cymbals/Bright Crash...,MN_Bright Crash_Crash_1112.4.wav,Crash,Bright Crash,Cymbals
3,./Samples/MDLib2.2/Sorted/Cymbals/Bright Crash...,ST_Bright Crash_Crash_1111.4.wav,Crash,Bright Crash,Cymbals
4,./Samples/MDLib2.2/Sorted/Cymbals/Bright Crash...,ST_Bright Crash_Crash_1111.3.wav,Crash,Bright Crash,Cymbals
...,...,...,...,...,...
3131,./Samples/MDLib2.2/Sorted/Kick SNoff/Long Kick...,IN_Long Kick_Release_0113.4.wav,Release,Long Kick,Kick SNoff
3132,./Samples/MDLib2.2/Sorted/Kick SNoff/Long Kick...,MN_Long Kick_Release_0113.4.wav,Release,Long Kick,Kick SNoff
3133,./Samples/MDLib2.2/Sorted/Kick SNoff/Long Kick...,DI_Long Kick_Release_0113.3.wav,Release,Long Kick,Kick SNoff
3134,./Samples/MDLib2.2/Sorted/Kick SNoff/Long Kick...,MN_Long Kick_Release_0111.5.wav,Release,Long Kick,Kick SNoff


In [27]:
rules = {
    'class_Snare': 'snare',
    'class_Tom': 'tom',
    'class_Cymbal': 'crash|ride',
    'subclass_Cymbal_Crash': 'crash',
    'subclass_Cymbal_Ride': 'ride',
    'class_Hat': 'HiHat',
    'class_Kick': 'Kick',
           }

set_classes(mdlib2_df, 'dir_2', rules)
mdlib2_df['dataset'] = 'MDLib'

In [28]:
mask = mdlib2_df.loc[:, [c for c in mdlib2_df.columns if c.startswith('class_')] ].sum(axis=1) < 1
print('unlabelled:', len(mdlib2_df[mask]))
print('labelled:', len(mdlib2_df[~mask]))
print('unlabelled:', list(mdlib2_df[mask]['dir_2']))

unlabelled: 0
labelled: 3136
unlabelled: []


In [62]:
mask = mdlib2_df.loc[:, [c for c in mdlib2_df.columns if c.startswith('class_')] ].sum(axis=1) > 1
print('overlabelled:', len(mdlib2_df[mask]))
print('overlabelled:', list(mdlib2_df[mask]['filename']))

overlabelled: 0
overlabelled: []


In [29]:
## IDMT

In [30]:
# idmt_df = DataFrame(data=idmt, columns=['path'])
# idmt_df.apply(split(),axis=1)

## Hydrogen Main Library

In [31]:
hydrogen_df = DataFrame(data=hydrogen, columns=['path'])
hydrogen_df = hydrogen_df.apply(split(2),axis=1)
hydrogen_df

,path,filename,dir_1
0,./Samples/Hydrogen/ForzeeStereo/CrashRide18Bel...,CrashRide18Bell-3.wav,ForzeeStereo
1,./Samples/Hydrogen/ForzeeStereo/Kick-4.wav,Kick-4.wav,ForzeeStereo
2,./Samples/Hydrogen/ForzeeStereo/TomHigh-4.wav,TomHigh-4.wav,ForzeeStereo
3,./Samples/Hydrogen/ForzeeStereo/Snare-0.wav,Snare-0.wav,ForzeeStereo
4,./Samples/Hydrogen/ForzeeStereo/SnareRim-2.wav,SnareRim-2.wav,ForzeeStereo
...,...,...,...
1002,./Samples/Hydrogen/VariBreaks/VP Kick 2.flac,VP Kick 2.flac,VariBreaks
1003,./Samples/Hydrogen/VariBreaks/VP Tom Low.flac,VP Tom Low.flac,VariBreaks
1004,./Samples/Hydrogen/VariBreaks/VP Drum.flac,VP Drum.flac,VariBreaks
1005,./Samples/Hydrogen/VariBreaks/VP Snare 1.flac,VP Snare 1.flac,VariBreaks


In [32]:
rules = {
    'class_Snare': 'snare|sn_|^sn|rim',
    'class_Hat': 'HiHat|hat|open|close|ohh|chh|Cl_HH|hh_p|Op_HH|hhpedal',
    'subclass_Hat_Open': 'close|chh|Cl_HH|Op_HH',
    'subclass_Hat_Close': 'open|ohh|hh_p|hhpedal',
    'class_Kick': 'kic?k|^bd',
    'class_Tom': '(?<!cus)(tom)(?!p)',
    'subclass_Snare_Rim': 'rim',
    'class_Cymbal': 'cra?sh|ride|china|splash|^rd',
    'subclass_Cymbal_Crash': 'cra?sh',
    'subclass_Cymbal_Ride': 'ride|^rd',
    'subclass_Cymbal_Chinese': 'china',
    'subclass_Cymbal_Splash': 'splash',
    'class_Bongo': 'bongo',
    'class_Agogo': 'agogo',
    'class_Clap': 'clap',
    'class_Cowbell': 'cow',
    'class_Djembe': 'djembe',
    'class_Conga': 'conga',
    'class_Clave': 'clave',
    'class_Shaken': 'shaker|maraca|cabasa',
    'subclass_Shaken_Shaker': 'shaker',
    'class_Tambourin': 'tambourin',
    'subclass_Shaken_Maracas': 'maraca',
    'subclass_Shaken_Cabasa': 'cabasa',
    'class_Cajon': 'cajon',
    'class_Timbales': 'timbale',
    'class_Guiro': 'guiro',
        }

set_classes(hydrogen_df, 'filename', rules)
hydrogen_df['dataset'] = 'Hydrogen'

In [33]:
mask = hydrogen_df.loc[:, [c for c in hydrogen_df.columns if c.startswith('class_')] ].sum(axis=1) < 1
print('unlabelled:', len(hydrogen_df[mask]))
print('labelled:', len(hydrogen_df[~mask]))
print('unlabelled:', list(hydrogen_df[mask]['filename']))

unlabelled: 225
labelled: 782
unlabelled: ['Stick-2.wav', 'Stick-3.wav', 'Stick-0.wav', 'Stick-1.wav', 'M-437 Gamelan 2.wav', 'S-145 Light Maple.wav', 'K-015 Dry 1.wav', 'K-087 909 Acoustic.wav', 'S-177 Reggae 4.wav', 'S-172 Real 4.wav', 'S-149 Memphis.wav', '808_sd.wav', '909_sd.wav', '808_mar.wav', '909_bd.wav', '808_bd.wav', 'hit_smear_1.wav', 'hit_smear_3.wav', '16269_ltibbits_HH_A_Custom_foot2.wav', '16270_ltibbits_HH_A_Custom_foot3_release.wav', '16268_ltibbits_HH_A_Custom_foot1.wav', 'Sdr158.wav', '16297_ltibbits_sticks_low_pitch.wav', 'Dununba1 Head Mute.wav', 'Sangban1 Head Mute.wav', 'Sangban1 Bell Mute.wav', 'Sangban1 Bell Hit.wav', 'Kenkeni1 Head Hit.wav', 'Kenkeni1 Head Mute.wav', 'Sangban1 Head Hit.wav', 'Kenkeni1 Bell Mute.wav', 'Kenkeni1 Bell Hit.wav', 'Dununba1 Head Hit.wav', 'Dununba1 Bell.wav', 'Dununba1 Bell Mute.wav', 'sim_k02-0.wav', '909_xOver_4a.wav', 'rhythm77_oh.wav', 'sim_bass_23_01.wav', 'rhythm77_bd.wav', 'fx_17b.wav', 'fx_09.wav', 'fx_10.wav', 'rhythm77_ch

In [34]:
mask = hydrogen_df.loc[:, [c for c in hydrogen_df.columns if c.startswith('class_')] ].sum(axis=1) > 1
print('overlabelled:', len(hydrogen_df[mask]))
print('overlabelled:', list(hydrogen_df[mask]['filename']))

overlabelled: 0
overlabelled: []


## Musicradar 1000

In [35]:
musicradar1000_df = DataFrame(data=musicradar1000, columns=['path'])
musicradar1000_df = musicradar1000_df.apply(split(3),axis=1)
musicradar1000_df

,path,filename,dir_1,dir_2
0,./Samples/musicradar-drum-samples/Drum Kits/Ki...,CYCdh_ElecK05-Snr04.wav,Kit 15 - Electro,Drum Kits
1,./Samples/musicradar-drum-samples/Drum Kits/Ki...,CYCdh_ElecK05-Kick02.wav,Kit 15 - Electro,Drum Kits
2,./Samples/musicradar-drum-samples/Drum Kits/Ki...,CYCdh_ElecK05-ClHat02.wav,Kit 15 - Electro,Drum Kits
3,./Samples/musicradar-drum-samples/Drum Kits/Ki...,CYCdh_ElecK05-Snr02.wav,Kit 15 - Electro,Drum Kits
4,./Samples/musicradar-drum-samples/Drum Kits/Ki...,CYCdh_ElecK05-ClHat03.wav,Kit 15 - Electro,Drum Kits
...,...,...,...,...
574,./Samples/musicradar-drum-samples/Assorted Hit...,CYCdh_MultiCrashHi-01.wav,Cymbals,Assorted Hits
575,./Samples/musicradar-drum-samples/Assorted Hit...,CYCdh_MultiCrashHi-02.wav,Cymbals,Assorted Hits
576,./Samples/musicradar-drum-samples/Assorted Hit...,CYCdh_Crash-02.wav,Cymbals,Assorted Hits
577,./Samples/musicradar-drum-samples/Assorted Hit...,CYCdh_Crash-01.wav,Cymbals,Assorted Hits


In [67]:
rules = {
    'class_Snare': 'snr|sdst|rim|flam|Brsh',
    'subclass_Snare_Off': 'snroff',
    'subclass_Snare_On': 'snron',
    'subclass_Snare_Side': 'sdst',
    'subclass_Snare_Rim': 'rim',
    'subclass_Snare_Flam': 'flam',
    'subclass_Snare_Brush': 'Brsh',
    'class_Hat': 'ClHat|OpHat|PdHat|hfhat',
    'subclass_Hat_Open': 'OpHat',
    'subclass_Hat_Close': 'ClHat|PdHat',
    'class_Kick': 'kick',
    'class_Tom': 'tom',
    'class_Cymbal': 'crash|trash|ride|cymbal|china',
    'subclass_Cymbal_Crash': 'crash',
    'subclass_Cymbal_Trash': 'trash',
    'subclass_Cymbal_Ride': 'ride',
    'subclass_Cymbal_Chinese': 'china',
    'class_Shaken': 'shkr',
    'subclass_Shaken_Shaker': 'shkr',
    'class_Tambourin': 'tamb',
    'class_Clap': 'clap'
        }

set_classes(musicradar1000_df, 'filename', rules)
musicradar1000_df['dataset'] = 'Musicradar1000'
# musicradar1000_df.loc[musicradar1000_df['class_Rim']==1, 'class_Snare']=0

In [66]:
mask = musicradar1000_df.loc[:, [c for c in musicradar1000_df.columns if c.startswith('class_')] ].sum(axis=1) < 1
print('unlabelled:', len(musicradar1000_df[mask]))
print('labelled:', len(musicradar1000_df[~mask]))
print('unlabelled:', list(musicradar1000_df.loc[mask,'filename']))

unlabelled: 135
labelled: 444
unlabelled: ['CYCdh_ElecK05-Snr04.wav', 'CYCdh_ElecK05-Snr02.wav', 'CYCdh_ElecK05-Snr01.wav', 'CYCdh_ElecK05-Snr03.wav', 'CYCdh_VinylK3-Perc03.wav', 'CYCdh_VinylK3-Perc04.wav', 'CYCdh_VinylK3-Perc02.wav', 'CYCdh_VinylK3-Snr01.wav', 'CYCdh_VinylK3-Snr02.wav', 'CYCdh_VinylK3-Perc01.wav', 'CyCdh_K3Snr-02.wav', 'CyCdh_K3Snr-07.wav', 'CyCdh_K3Snr-08.wav', 'CyCdh_K3Snr-05.wav', 'CyCdh_K3Snr-04.wav', 'CyCdh_K3Snr-09.wav', 'CyCdh_K3Snr-10.wav', 'CyCdh_K3Snr-01.wav', 'CyCdh_K3Snr-06.wav', 'CyCdh_K3Snr-03.wav', 'CYCdh_Kurz07-Snr01.wav', 'CYCdh_Kurz07-Perc03.wav', 'CYCdh_Kurz07-Perc01.wav', 'CYCdh_Kurz07-Snr02.wav', 'CYCdh_Kurz07-RevHat.wav', 'CYCdh_Kurz07-Snr03.wav', 'CYCdh_Kurz07-Snr04.wav', 'CYCdh_K4-Snr04.wav', 'CYCdh_K4-Snr03.wav', 'CYCdh_K4-Snr05.wav', 'CYCdh_K4-Snr07.wav', 'CYCdh_K4-Snr08.wav', 'CYCdh_K4-Snr10.wav', 'CYCdh_K4-Snr11.wav', 'CYCdh_K4-Snr01.wav', 'CYCdh_K4-Snr09.wav', 'CYCdh_K4-Snr06.wav', 'CYCdh_K4-Snr02.wav', 'CYCdh_VinylK2-Snr02.wav', 'CYCdh_Vi

In [38]:
mask = musicradar1000_df.loc[:, [c for c in musicradar1000_df.columns if c.startswith('class_')] ].sum(axis=1) > 1
print('overlabelled:', len(musicradar1000_df[mask]))
print('overlabelled:', list(musicradar1000_df[mask]['filename']))

overlabelled: 0
overlabelled: []


## Merge datasets

In [39]:
df = pd.concat([#idmt_df,
                #mdlib1_df,
                #mdlib2_df,
                drum200_df,
                wavegan_df,
                musicradar_df,
                musicradar1000_df,
                hydrogen_df],
               axis=0,
               sort=False,
               ignore_index=True
              )
df.shape

(11832, 43)

In [40]:
df[df[[c for c in df.columns if c.startswith('class_')]].sum(axis=1) > 1]

,path,filename,dir_1,class_Hat,subclass_Hat_Open,subclass_Hat_Close,class_Bongo,class_Cymbal,subclass_Cymbal_Crash,subclass_Cymbal_Ride,...,class_Cajon,dir_2,subclass_Snare_Off,subclass_Snare_On,subclass_Snare_Side,subclass_Snare_Flam,subclass_Snare_Brush,subclass_Cymbal_Trash,class_Timbale,class_Guiro
1659,./Samples/200-drum-machines/Fairlight IIx/MaxV...,MaxV - HHTamb.wav,Fairlight IIx,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2325,./Samples/200-drum-machines/Kawai r-50/MaxV - ...,MaxV - ATOMBDEN.wav,Kawai r-50,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2341,./Samples/200-drum-machines/Kawai r-50/MaxV - ...,MaxV - ATOMBDRV.wav,Kawai r-50,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2378,./Samples/200-drum-machines/Kawai r-50/MaxV - ...,MaxV - ATOMBDGA.wav,Kawai r-50,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4010,./Samples/200-drum-machines/Roland SC-88/MaxV ...,MaxV - D#4 Open High Conga.wav,Roland SC-88,1.0,1.0,1.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4164,./Samples/200-drum-machines/Sequential Tom/TOM...,TOMSnare.wav,Sequential Tom,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4166,./Samples/200-drum-machines/Sequential Tom/TOM...,TOMHat_O.wav,Sequential Tom,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4169,./Samples/200-drum-machines/Sequential Tom/TOM...,TOMClap.wav,Sequential Tom,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4177,./Samples/200-drum-machines/Sequential Tom/TOM...,TOMCymbal.wav,Sequential Tom,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4181,./Samples/200-drum-machines/Sequential Tom/TOM...,TOMKick.wav,Sequential Tom,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
#df[df[[c for c in df.columns if c.startswith('class_')]].sum(axis=1) < 1]
overlabelled = df[[c for c in df.columns if c.startswith('class_')]].sum(axis=1) > 1
list(df[overlabelled]['filename'])

['MaxV - HHTamb.wav',
 'MaxV - ATOMBDEN.wav',
 'MaxV - ATOMBDRV.wav',
 'MaxV - ATOMBDGA.wav',
 'MaxV - D#4 Open High Conga.wav',
 'TOMSnare.wav',
 'TOMHat_O.wav',
 'TOMClap.wav',
 'TOMCymbal.wav',
 'TOMKick.wav',
 'TOMHat_C.wav',
 'MaxV - RIM-TOM3.wav',
 'MaxV - RIM-TOM1.wav',
 'MaxV - RIM-TOM2.wav',
 'MaxV - RIM-TOM4.wav',
 'Shaken Tamb 05.wav',
 'Shaken Tamb 02.wav',
 'Shaken Tamb 04.wav',
 'Shaken Tamb 03.wav',
 'Shaken Tamb 01.wav']

In [42]:
df.shape

(11832, 43)

## Cleaning

### Anomalies

In [102]:
clean = df[[c for c in df.columns if c.startswith('class_')]].sum(axis=1) == 1
df_clean = df[clean]
classes = [c for c in df.columns if c.startswith('class_')]
subclasses = [c for c in df.columns if c.startswith('subclass_')]
df_clean.loc[:,classes + subclasses] = df_clean.loc[:,classes + subclasses].fillna(0)

In [103]:
df_clean

,path,filename,dir_1,class_Hat,subclass_Hat_Open,subclass_Hat_Close,class_Bongo,class_Cymbal,subclass_Cymbal_Crash,subclass_Cymbal_Ride,...,class_Cajon,dir_2,subclass_Snare_Off,subclass_Snare_On,subclass_Snare_Side,subclass_Snare_Flam,subclass_Snare_Brush,subclass_Cymbal_Trash,class_Timbale,class_Guiro
0,./Samples/200-drum-machines/Yamaha RX-11/MaxV ...,MaxV - RX11 OPHAT2.wav,Yamaha RX-11,1.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,./Samples/200-drum-machines/Yamaha RX-11/MaxV ...,MaxV - RX11 BD3.wav,Yamaha RX-11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,./Samples/200-drum-machines/Yamaha RX-11/MaxV ...,MaxV - RX11 BD2.wav,Yamaha RX-11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,./Samples/200-drum-machines/Yamaha RX-11/MaxV ...,MaxV - RX11 CLHAT2.wav,Yamaha RX-11,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,./Samples/200-drum-machines/Yamaha RX-11/MaxV ...,MaxV - RX11 RIDE.wav,Yamaha RX-11,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11826,./Samples/Hydrogen/VariBreaks/VP Hat 3 Cl.flac,VP Hat 3 Cl.flac,VariBreaks,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11827,./Samples/Hydrogen/VariBreaks/VP Kick 2.flac,VP Kick 2.flac,VariBreaks,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11828,./Samples/Hydrogen/VariBreaks/VP Tom Low.flac,VP Tom Low.flac,VariBreaks,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11830,./Samples/Hydrogen/VariBreaks/VP Snare 1.flac,VP Snare 1.flac,VariBreaks,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [64]:
# timbre = timbral_models.timbral_extractor(df_clean['path'][0])
# timbre.values()

In [74]:
mfccs = []
for i, index in enumerate(df_clean.index):
    file = df_clean.loc[index,'path']
    sound, _ = load(file, sr=SR)
    sound = fix_length(sound, 1024*48)
    feat = feature.mfcc(sound, sr=SR, n_mels=20)
    mfccs.append(feat)
    df_clean.loc[index, 'mfcc_index'] = i
df_clean['mfcc_index'] = df_clean['mfcc_index'].astype(int)
mfccs = np.stack(mfccs)

In [79]:
SR=22050
for _class in classes:
    files = df_clean.loc[df_clean[_class]==1,'path']
    mfcc_index = df_clean.loc[df_clean[_class]==1,'mfcc_index']
    mfcc = mfccs[mfcc_index]
    mfcc = mfcc.reshape(mfcc.shape[0], -1)
    reducer = umap.UMAP()
    embedding = reducer.fit_transform(mfcc)
    print(_class)
    display_embedding(
        embedding[:, 0],
        embedding[:, 1],
        df_clean.loc[df_clean[_class]==1,'dataset'].values,
        files
    )

class_Hat


    'data': [{'customdata': array([['./Samples/200-drum-machines/Yamaha RX-11/Ma…

class_Bongo


    'data': [{'customdata': array([['./Samples/200-drum-machines/Alesis Sr16/SR1…

class_Cymbal


...

class_Timbale


    'data': [{'customdata': array([['./Samples/Hydrogen/ErnysPercussion/Timbale …

/home/guillaume/miniconda3/envs/DrumAI/lib/python3.7/site-packages/umap/umap_.py:1679: UserWarning:

n_neighbors is larger than the dataset size; truncating to X.shape[0] - 1



class_Guiro


    'data': [{'customdata': array([['./Samples/Hydrogen/ErnysPercussion/1Guiro L…

In [45]:
unlabelled = df[[c for c in df.columns if c.startswith('class_')]].sum(axis=1) < 1
print(len(list(df[unlabelled]['filename'])))
list(df[unlabelled]['filename'])

3775


['MaxV - RX11 MED SN.wav',
 'MaxV - RX11 LITE SN.wav',
 'MaxV - RX11 HEVY SN.wav',
 'MaxV - Metal.wav',
 '13.wav',
 '06.wav',
 '05.wav',
 '09.wav',
 '08.wav',
 '01.wav',
 '00.wav',
 '14.wav',
 '11.wav',
 '02.wav',
 '12.wav',
 '03.wav',
 '10.wav',
 '04.wav',
 '07.wav',
 '15.wav',
 'SR16Tank.wav',
 'SR16FSnap.wav',
 'SR16Guiro.wav',
 'SR16Spark.wav',
 'SR16Block.wav',
 'Acoustic Bass Drum.wav',
 'MaxV - EMU14.wav',
 'MaxV - KDR21.wav',
 'MaxV - EMU13.wav',
 'MaxV - EMU20.wav',
 'MaxV - EMU19.wav',
 'MaxV - DRM03.wav',
 'MaxV - SND24.wav',
 'MaxV - EMU15.wav',
 'MaxV - EMU09.wav',
 'MaxV - DRM04.wav',
 'MaxV - EMU12.wav',
 'MaxV - CLD01.wav',
 'MaxV - DRM08.wav',
 'MaxV - DRM07.wav',
 'MaxV - EMU17.wav',
 'MaxV - OHD22.wav',
 'MaxV - EMU16.wav',
 'MaxV - EMU18.wav',
 'MaxV - EMU_SD.wav',
 'MaxV - DRM05.wav',
 'MaxV - EMU11.wav',
 'DrumulatorBlock.wav',
 'MaxV - EMU10.wav',
 'MaxV - DRM06.wav',
 'MaxV - SND23.wav',
 'MaxV - CLD02.wav',
 'MaxV - Snap.wav',
 '06.wav',
 '05.wav',
 '01.wav',
 

In [113]:
def change_class(df, filename, new_class, sub_class=None):
    df.loc[df['filename']==filename, classes] = 0
    df.loc[df['filename']==filename, subclasses] = 0
    df.loc[df['filename']==filename, new_class] = 1
    if sub_class is not None:
        df.loc[df['filename']==filename, sub_class] = 1

In [119]:
overlabelled = df[[c for c in df.columns if c.startswith('class_')]].sum(axis=1) > 1
print(len(list(df[overlabelled]['filename'])))
list(df[overlabelled]['filename'])
for path, file in zip(df[overlabelled]['path'], df[overlabelled]['filename']):
    sound, sr = load(path)
    print(file)
    IPython.display.display(IPython.display.Audio(sound, rate=sr, autoplay=False))

0


In [118]:
change_class(df, 'MaxV - HHTamb.wav', 'class_Tambourin')
for file in ['MaxV - ATOMBDEN.wav', 'MaxV - ATOMBDRV.wav', 'MaxV - ATOMBDGA.wav']:
    change_class(df, file, 'class_Kick')
change_class(df, 'TOMSnare.wav', 'class_Snare')
change_class(df, 'TOMHat_O.wav', 'class_Hat', 'subclass_Hat_Open')
change_class(df, 'TOMClap.wav', 'class_Clap')
change_class(df, 'TOMCymbal.wav', 'class_Cymbal')
change_class(df, 'TOMKick.wav', 'class_Kick')
change_class(df, 'TOMHat_C.wav', 'class_Hat', 'subclass_Hat_Close')
for file in ['MaxV - RIM-TOM1.wav', 'MaxV - RIM-TOM2.wav','MaxV - RIM-TOM3.wav','MaxV - RIM-TOM4.wav']:
    change_class(df, file, 'class_Snare', 'subclass_Snare_Rim')
for file in ['Shaken Tamb 01.wav', 'Shaken Tamb 02.wav', 'Shaken Tamb 03.wav', 'Shaken Tamb 04.wav', 'Shaken Tamb 05.wav']:
    change_class(df, file, 'class_Tambourin')

In [116]:
classes = [c for c in df.columns if c.startswith('class_')]
df_clean.loc[:,classes].sum().sort_values()

class_Darbuka         4.0
class_Gong            4.0
class_Timbale         4.0
class_Guiro           6.0
class_Triangle        8.0
class_Tabla           9.0
class_Cajon          12.0
class_Djembe         20.0
class_Agogo          47.0
class_Timpani        50.0
class_Clave          54.0
class_Bongo          60.0
class_Tambourin      75.0
class_Shaken        104.0
class_Conga         120.0
class_Cowbell       152.0
class_Clap          307.0
class_Cymbal        722.0
class_Tom          1268.0
class_Hat          1289.0
class_Kick         1556.0
class_Snare        2166.0
dtype: float64

In [48]:
classes = [c for c in df.columns if c.startswith('subclass_')]
df_clean.loc[:,classes].sum().sort_values()

subclass_Snare_On            0.0
subclass_Snare_Brush         3.0
subclass_Snare_Flam         10.0
subclass_Cymbal_Trash       10.0
subclass_Shaken_Maracas     11.0
subclass_Snare_Off          16.0
subclass_Shaken_Cabasa      32.0
subclass_Snare_Side         33.0
subclass_Cymbal_Splash      34.0
subclass_Cymbal_Chinese     35.0
subclass_Shaken_Shaker      49.0
subclass_Snare_Rim         186.0
subclass_Cymbal_Ride       275.0
subclass_Cymbal_Crash      297.0
subclass_Hat_Open          366.0
subclass_Hat_Close         397.0
dtype: float64

In [35]:
df.to_csv('full.csv')

In [49]:
from random import sample
classes_sum = df_clean.loc[:,classes].sum()
column_selection = classes_sum[classes_sum > 100].index
#column_selection
df_sample = DataFrame()
for col in column_selection:
    selection = (df_clean[col] == 1)
    index = df_clean[selection].index
    if len(index) > 200:
        index = sample(list(index), 200)
    df_sample = pd.concat( [df_sample, df_clean.loc[index, ['path'] + list(column_selection)]])
df_sample = df_sample.fillna(0)

In [50]:
df_sample
#df_sample.loc[:,column_selection].sum().sort_values()

,path,subclass_Hat_Open,subclass_Hat_Close,subclass_Cymbal_Crash,subclass_Cymbal_Ride,subclass_Snare_Rim
2452,./Samples/200-drum-machines/MXR Computer Rhyth...,1.0,1.0,0.0,0.0,0.0
5115,./Samples/200-drum-machines/AKAI XE-8/MaxV - X...,1.0,1.0,0.0,0.0,0.0
10184,./Samples/musicradar-essential-drumkit-samples...,1.0,0.0,0.0,0.0,0.0
272,./Samples/200-drum-machines/Roland Tr-606/TR60...,1.0,0.0,0.0,0.0,0.0
6639,./Samples/200-drum-machines/Melosonic/MaxV - H...,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...
11723,./Samples/Hydrogen/Classic-626/Classic-626_Rim...,0.0,0.0,0.0,0.0,1.0
11736,./Samples/Hydrogen/ColomboAcousticDrumkit/snar...,0.0,0.0,0.0,0.0,1.0
11744,./Samples/Hydrogen/ColomboAcousticDrumkit/stic...,0.0,0.0,0.0,0.0,1.0
11745,./Samples/Hydrogen/ColomboAcousticDrumkit/snar...,0.0,0.0,0.0,0.0,1.0


In [38]:
df_sample.to_csv('data.csv')